# Spark SQL

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Laboratorio 1 PySpark") \
    .getOrCreate()

sc = spark.sparkContext

In [76]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np

# Helper function to test the correctness of the solutions
def test(var, val, msg=""):
    print("1 test passed.") if var == val else print("1 test failed. " + msg)

## Ejercicio 0

In [3]:
from pyspark import Row
from pyspark.sql import functions as sql_f
lines_df = spark.read.text('C:\\Users\\ruben\\Downloads\\quixote.txt')


In [4]:
def word_count(ruta):
    lines_df = spark.read.text(ruta)
    ordenado = lines_df.select(sql_f.explode(sql_f.split(lines_df.value, ' '))\
        .alias('word'))\
        .filter("word != ''")\
        .groupBy('word')\
        .count()\
        .sort('count', ascending = False)
    return ordenado.take(10)

In [5]:
word_count('C:\\Users\\ruben\\Downloads\\quixote.txt')

[Row(word='the', count=20923),
 Row(word='and', count=16606),
 Row(word='to', count=13492),
 Row(word='of', count=12866),
 Row(word='that', count=7164),
 Row(word='a', count=7003),
 Row(word='in', count=6860),
 Row(word='I', count=5756),
 Row(word='he', count=5640),
 Row(word='for', count=4534)]

In [7]:
top10_quixote = word_count('C:\\Users\\ruben\\Downloads\\quixote.txt')
print(top10_quixote)
test(top10_quixote, [('the', 20923), ('and', 16606), ('to', 13492), ('of', 12866), 
                                  ('that', 7164), ('a', 7003), ('in', 6860), ('I', 5756), ('he', 5640), 
                                  ('for', 4534)], "Try again!")

[Row(word='the', count=20923), Row(word='and', count=16606), Row(word='to', count=13492), Row(word='of', count=12866), Row(word='that', count=7164), Row(word='a', count=7003), Row(word='in', count=6860), Row(word='I', count=5756), Row(word='he', count=5640), Row(word='for', count=4534)]
1 test passed.


# Ejercicio 1

In [26]:
def remove_punctuation(column):
    # Remove punctuation using regexp_replace, then trim and convert to lowercase
    return lower(trim(regexp_replace(regexp_replace("sentence", r'[^a-zA-Z0-9\s]', ' '), r'\s+', ' ')))
import pyspark.sql.functions as sql_f # import SQL functions
from pyspark.sql.functions import regexp_replace, lower, trim


In [11]:
df_text_example = spark.createDataFrame([(u'Hi! How are you?',),
                                         (u' Removing_all*kind^of+punctuations and extra       spaces     ',)], ['sentence'])
df_text_example.show(truncate=False)

+--------------------------------------------------------------+
|sentence                                                      |
+--------------------------------------------------------------+
|Hi! How are you?                                              |
| Removing_all*kind^of+punctuations and extra       spaces     |
+--------------------------------------------------------------+



In [22]:
df_text_example.select(remove_punctuation(sql_f.col("sentence"))).show(truncate=False)

+--------------------------------------------------------------------------------------+
|lower(trim(regexp_replace(regexp_replace(sentence, [^a-zA-Z0-9\s],  , 1), \s+,  , 1)))|
+--------------------------------------------------------------------------------------+
|hi how are you                                                                        |
|removing all kind of punctuations and extra spaces                                    |
+--------------------------------------------------------------------------------------+



In [74]:
from pyspark.sql.functions import explode, col, lower, regexp_replace, split, trim

def remove_punctuation(column):
    return lower(trim(regexp_replace(regexp_replace(column, r'[^a-zA-Z0-9\s]', ' '), r'\s+', ' ')))

def word_count2(ruta):
    lines_df = spark.read.text(ruta)
    words_df = lines_df.select(explode(split(remove_punctuation(col("value")), ' ')).alias('word'))
    words_df = words_df.filter("word != ''")
    word_counts = words_df.groupBy('word').count()
    ordered_word_counts = word_counts.orderBy('count', ascending=False)
    top_10_words = ordered_word_counts.take(10)
    return top_10_words
    
word_count2('C:\\Users\\ruben\\Downloads\\quixote.txt')

[Row(word='the', count=22471),
 Row(word='and', count=17722),
 Row(word='to', count=14006),
 Row(word='of', count=13491),
 Row(word='that', count=7993),
 Row(word='in', count=7335),
 Row(word='a', count=7287),
 Row(word='i', count=6653),
 Row(word='he', count=6157),
 Row(word='it', count=5680)]

In [77]:
top10_quixote = word_count2('C:\\Users\\ruben\\Downloads\\quixote.txt')
test(top10_quixote, [('the', 22471), ('and', 17722), ('to', 14006), ('of', 13491), ('that', 7993),
                                 ('in', 7335), ('a', 7287), ('i', 6653), ('he', 6157), ('it', 5680)],
                  'Try again!')

1 test passed.


# Ejercicio 2. Removing stop words

In [78]:
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 
              'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 
              'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into',
              'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 
              'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below',
              'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me',
              'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 
              'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she',
              'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and',
              'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 
              'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not',
              'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too',
              'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't',
              'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 
              'how', 'further', 'was', 'here', 'than']

In [82]:
from pyspark.sql.functions import explode, col, lower, regexp_replace, split, trim

def remove_punctuation(column):
    return lower(trim(regexp_replace(regexp_replace(column, r'[^a-zA-Z0-9\s]', ' '), r'\s+', ' ')))

def word_count2(ruta):
    lines_df = spark.read.text(ruta)
    words_df = lines_df.select(explode(split(remove_punctuation(col("value")), ' ')).alias('word'))
    words_df = words_df.filter((col("word") != "") & (~col("word").isin(stop_words))) #chatgpt
    word_counts = words_df.groupBy('word').count()
    ordered_word_counts = word_counts.orderBy('count', ascending=False)
    top_10_words = ordered_word_counts.take(10)
    return top_10_words
    
word_count2('C:\\Users\\ruben\\Downloads\\quixote.txt')

[Row(word='said', count=2627),
 Row(word='quixote', count=2327),
 Row(word='sancho', count=2205),
 Row(word='one', count=1623),
 Row(word='would', count=1251),
 Row(word='thou', count=1229),
 Row(word='say', count=904),
 Row(word='good', count=889),
 Row(word='may', count=855),
 Row(word='see', count=774)]

In [83]:
top10_quixote = word_count2('C:\\Users\\ruben\\Downloads\\quixote.txt')
test(top10_quixote, [('said', 2627), ('quixote', 2327), ('sancho', 2205), ('one', 1623), ('would', 1251),
                                  ('thou', 1229), ('say', 904), ('good', 889), ('may', 855), ('see', 774)],
                  'Try again!')

1 test passed.


# Histograma de la repeticion de palabras.

In [ ]:
def histogram_reps(file_path):
    lines_df = spark.read.text(ruta)
    words_df = lines_df.select(explode(split(remove_punctuation(col("value")), ' ')).alias('word'))
histogram_reps('C:\\Users\\ruben\\Downloads\\quixote.txt')[:20]

In [88]:
lines_df = spark.read.text(ruta)
words_df = lines_df.select(explode(split(remove_punctuation(col("value")), ' ')).alias('word'))
words_df = words_df.filter((col("word") != "") & (~col("word").isin(stop_words))) #chatgpt
words_counts = words_df.groupBy('word').count()
words_counts.sql_f

+-----------+-----+
|       word|count|
+-----------+-----+
|     online|    4|
|  imitation|   12|
|      still|  235|
|     doubts|   13|
|       hope|   75|
|   everyday|    3|
|transmitted|    1|
| superseded|    1|
|     leonor|    1|
|     poetry|   23|
|       1572|    1|
|      turks|   23|
|     ransom|   21|
| concluding|    2|
|     freaks|    4|
|        art|  266|
|    insipid|    1|
|ingratitude|   10|
|      inner|    1|
|      besom|    1|
+-----------+-----+
only showing top 20 rows

